# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Импортируем необходимые библиотеки:

In [ ]:
import pandas as pd # импорт библиотеки pandas
import matplotlib.pyplot as plt # импорт библиотеки matplotlib
plt.style.use('seaborn-pastel')
import seaborn as sns # импорт библиотеки seaborn
import numpy as np # импорт библиотеки nump
from scipy import stats as st # импорт библиотеки scipy
from sklearn.linear_model import LinearRegression # импорт линейной регрессии
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split # импорт разделителя
from sklearn.utils import shuffle # импорт шаффла для перемешивания
import warnings
warnings.filterwarnings("ignore")

In [ ]:
reg_1 = pd.read_csv('/datasets/geo_data_0.csv')
reg_2 = pd.read_csv('/datasets/geo_data_1.csv')
reg_3 = pd.read_csv('/datasets/geo_data_2.csv')

#создадим словарь с регионами
region_list = {'Region 1': reg_1,
              'Region 2': reg_2,
              'Region 3': reg_3}

In [ ]:
def inf(data):
    display(data.head())
    display(data.info())
    print('Количество пропусков')
    display(data.isnull().sum())
    print('Количество дубликатов')
    display(data.duplicated().sum())

### reg_1

In [ ]:
inf(reg_1)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Количество пропусков


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Количество дубликатов


0

In [ ]:
print(reg_1['id'].duplicated().sum())
print(reg_1['f0'].duplicated().sum())
print(reg_1['f1'].duplicated().sum())
print(reg_1['f2'].duplicated().sum())

10
0
0
0


С данными по первому региону все в порядке, кроме столбца 'id'.

### reg_2

In [ ]:
inf(reg_2)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Количество пропусков


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Количество дубликатов


0

In [ ]:
print(reg_2['id'].duplicated().sum())
print(reg_2['f0'].duplicated().sum())
print(reg_2['f1'].duplicated().sum())
print(reg_2['f2'].duplicated().sum())

4
0
0
0


Второй регион тоже без замечаний, кроме 'id' - также обнаружены пропуски.

### reg_3

In [ ]:
inf(reg_3)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Количество пропусков


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Количество дубликатов


0

In [ ]:
print(reg_3['id'].duplicated().sum())
print(reg_3['f0'].duplicated().sum())
print(reg_3['f1'].duplicated().sum())
print(reg_3['f2'].duplicated().sum())

4
0
0
0


Третий регион явных недостатков тоже не имеет, кроме 'id'.

**Вывод**

Для построения будущей модели столбец 'id' явно не несет большой ценности, так как соответственно, удалим его из всех датафреймов.

In [ ]:
reg_1 = reg_1.drop('id', axis = 1)
reg_2 = reg_2.drop('id', axis = 1)
reg_3 = reg_3.drop('id', axis = 1)

## Обучение и проверка модели

Для каждого региона обучим и проверим модель линейной регрессии.

* разобьем выборку на train и vail в соотношении 75 к 25
* обучим модель и сделаем предсказания на valid
* сохраним предсказания и правильные ответы на valid
* выведем на экран средний запас предсказанного сырья и RMSE модели
* проанализируем полученные результаты

Создадим для удобства функцию для подсчета RMSE:

In [ ]:
def rmse(target_valid, predictions_valid):
    x = mean_squared_error(target_valid, predictions_valid)**0.5
    return x

Создадим для удобства функцию, которая сразу будет
* выделять целевой признак
* выделять параметры
* делить выборку
* обучать модель
* подсчитывать RMSE

На вход она, соответственно, будет принимать датасет.

In [ ]:
def model_calc(data):
    #features и target
    features = data.drop('product', axis = 1)
    target = data['product']
    #делим на valid и train
    features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                                 target,
                                                                                 test_size = 0.25,
                                                                                 random_state = 12345)
    #обучаем модель
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    #rmse
    rmse_data = rmse(target_valid, predictions)
    return target_valid, predictions, rmse_data

In [ ]:
reg_1_tg_valid, predictions_1_valid, rmse_1 = model_calc(reg_1)
reg_2_tg_valid, predictions_2_valid, rmse_2 = model_calc(reg_2)
reg_3_tg_valid, predictions_3_valid, rmse_3 = model_calc(reg_3)

Посчитаем средний предсказанный запас сырья:

In [ ]:
mean_product_1 = predictions_1_valid.mean()
mean_product_2 = predictions_2_valid.mean()
mean_product_3 = predictions_3_valid.mean()

Изучим результат:

In [ ]:
print('Средний запас предсказанного сырья для 1го региона:', round(mean_product_1, 2), 'RMSE:', rmse_1)
print('Средний запас предсказанного сырья для 2го региона:', round(mean_product_2, 2), 'RMSE:', rmse_2)
print('Средний запас предсказанного сырья для 3го региона:', round(mean_product_3, 2), 'RMSE:', rmse_3)

Средний запас предсказанного сырья для 1го региона: 92.59 RMSE: 37.5794217150813
Средний запас предсказанного сырья для 2го региона: 68.73 RMSE: 0.893099286775617
Средний запас предсказанного сырья для 3го региона: 94.97 RMSE: 40.02970873393434


**Вывод**

Лучший параметр RMSE показывает модель на 2м регионе.

## Подготовка к расчёту прибыли

Создадим необходимые константы, в которые вложим известные нам значения:

In [ ]:
n_bores = 500
n_exp = 200
capital = 10000000000 / 1000000
cost = 450000 / 1000000
risk = 2.5

Проведем подсчеты:

In [ ]:
product_needed = capital / n_exp / cost

print('Необходимый средний объем одной скважины, чтобы она была неубыточна:', product_needed)
print('')
for reg in region_list:
    x = region_list[reg]['product'].mean()
    if x > product_needed:
        print(f'Средний объем сырья в {reg} неубыточный: {round(x, 3)}')
    else:
        print(f'Средний объем сырья в {reg} убыточен: {round(x, 3)}')

Необходимый средний объем одной скважины, чтобы она была неубыточна: 111.11111111111111

Средний объем сырья в Region 1 убыточен: 92.5
Средний объем сырья в Region 2 убыточен: 68.825
Средний объем сырья в Region 3 убыточен: 95.0


**Вывод**

По полученным данным видим, что все регионы убыточны.

## Расчёт прибыли и рисков

Создадим функцию, которая будет расчитывать прибыль по выбранным скважинам, а также предсказаниям модели.

* Выберем скважины с максимальными значениями предсказаний
* Просуммироуем целевое значение объема сырья, соответствующие этим предсказаниям
* Рассчитаем прибыль для полученного объема сырья

In [ ]:
#преобразуем результаты предскзааний в Pandas Series

predictions_1_valid = pd.Series(predictions_1_valid)
predictions_2_valid = pd.Series(predictions_2_valid)
predictions_3_valid = pd.Series(predictions_3_valid)

In [ ]:
def revenue_profit (target, bores, count, capital):
    boes_sorted = bores.sort_values(ascending = False) # сортируем по предсказанию
    selected = target[boes_sorted.index][:count] # выбираем лучшие но по фактическим индексам
    revenue = cost * selected.sum() # считаем заработок с выбранных скважин
    profit = revenue - capital #считаем прибыль
    return profit

Посчитаем риски и прибыль для каждого региона:

1. Применим технику Booystrap с 1000 выборок, чтобы найти распределение прибыли
2. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток - это отрицательная прибыль.
3. Подготовим выводы - предложим регион для разработки скважин и обоснуем выбор.

Создадим функцию для подсчета результатов:

In [ ]:
def check_region(target, predictions):
    #создаем список
    values = []
    #применим Bootstrap
    state = np.random.RandomState(12345)
    for i in range(1000):
        target_subsample = target.reset_index(drop = True).sample(n = n_bores, replace = True, random_state = state)
        pred_subsample = predictions[target_subsample.index]
        value = revenue_profit(target_subsample, pred_subsample, n_exp, capital)
        values.append(value)
    #считаем количество убытков
    values = pd.Series(values)
    risk = (values < 0).mean()
    #считаем доверительный интервал
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    #выводим результат
    print('Средняя прибыль в регионе:', round(values.mean(), 2), 'млн.руб.')
    print(f'95% доверительный интервал: ({lower}, {upper})')
    print('Риск убытков: {:1%}'.format((values < 0).mean()))
    #сравниваем с необходимым риском
    if (risk * 100) <= 2.5:
        print('Риск убытков меньше необходимого значения. Регион можно оставить.')
    else:
        print('Риск убытков больше необходимого значения.')

Первый регион:

In [ ]:
check_region(reg_1_tg_valid, predictions_1_valid)

Средняя прибыль в регионе: 425.94 млн.руб.
95% доверительный интервал: (-102.09009483793592, 947.976353358369)
Риск убытков: 6.000000%
Риск убытков больше необходимого значения.


Второй регион:

In [ ]:
check_region(reg_2_tg_valid, predictions_2_valid)

Средняя прибыль в регионе: 515.22 млн.руб.
95% доверительный интервал: (68.87322537050254, 931.547591257049)
Риск убытков: 1.000000%
Риск убытков меньше необходимого значения. Регион можно оставить.


Третий регион:

In [ ]:
check_region(reg_3_tg_valid, predictions_3_valid)

Средняя прибыль в регионе: 435.01 млн.руб.
95% доверительный интервал: (-128.88054732978893, 969.7069541802657)
Риск убытков: 6.400000%
Риск убытков больше необходимого значения.


**Вывод**

В ходе проведенного исследования мы получили результат, что у первого и третьего региона средняя прибыль меньше, чем у второго региона, а также вероятность убытков выше, соответственно, их рассматривать не имеет смысла.

Для дальнейших установок скважин наиболее подходящим вариантом является второй регион, так как он показывает самый изкий риск убытков и самую высокую среднюю прибыль из трех вариантов.